In [ ]:
"""libraries required"""
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install timm
!pip install torchattacks

In [2]:
"""connect to google drive to store and retrieve data"""
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import h5py
import torch
from torch.utils.data import TensorDataset

device = "cuda" if torch.cuda.is_available() else "cpu"

data_list = []
label_list = []

# Open the HDF5 file
h5_file_path = '/content/drive/MyDrive/data/AdvData/ESC50.h5'
with h5py.File(h5_file_path, 'r') as hf:
    # Iterate over the samples in the HDF5 file
    for sample_name in hf.keys():
        data = torch.tensor(hf[sample_name]['data'][:])
        label = hf[sample_name]['label'][()]

        # Append data and label to lists
        data_list.append(data)
        label_list.append(label)

esc = TensorDataset(torch.stack(data_list), torch.tensor(label_list))

In [5]:
#filename for ESC50
# 1-100032-A-0.wav
# target 0 category dog
img1 = esc[0]
print(img1)

(tensor([[[5.4443e-08, 5.4443e-08, 5.4443e-08,  ..., 5.4443e-08,
          5.4443e-08, 5.4443e-08],
         [5.4443e-08, 5.4443e-08, 5.4443e-08,  ..., 5.4443e-08,
          5.4443e-08, 5.4443e-08],
         [5.4443e-08, 5.4443e-08, 5.4443e-08,  ..., 5.4443e-08,
          5.4443e-08, 5.4443e-08],
         ...,
         [5.4443e-08, 5.4443e-08, 5.4443e-08,  ..., 5.4443e-08,
          5.4443e-08, 5.4443e-08],
         [5.4443e-08, 5.4443e-08, 5.4443e-08,  ..., 5.4443e-08,
          5.4443e-08, 5.4443e-08],
         [5.4443e-08, 5.4443e-08, 5.4443e-08,  ..., 5.4443e-08,
          5.4443e-08, 5.4443e-08]]]), tensor(0))


In [6]:
import timm
import torch.nn as nn

In [30]:
from skimage.metrics import structural_similarity as ssim
import numpy as np
import torchattacks

def cal_All(im, out_feat, datasetName):
    img1 = im[0].unsqueeze(0)
    label = im[1].unsqueeze(0)
    resnet18 = timm.create_model('resnet18.a1_in1k', pretrained=True)
    resnet18.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    resnet18.fc = nn.Linear(in_features=512, out_features=out_feat, bias=True)
    torch.cuda.empty_cache()
    checkpoint = torch.load(f'/content/drive/MyDrive/data/Models/{datasetName}/resnet18.pth')
    resnet18.load_state_dict(checkpoint)

    resnet50 = timm.create_model('resnet50.a1_in1k', pretrained=True)
    resnet50.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    resnet50.fc = nn.Linear(in_features=2048, out_features=out_feat, bias=True)
    torch.cuda.empty_cache()
    checkpoint = torch.load(f'/content/drive/MyDrive/data/Models/{datasetName}/resnet50.pth')
    resnet50.load_state_dict(checkpoint)

    vit_base = timm.create_model('vit_base_patch16_224.augreg2_in21k_ft_in1k', pretrained=True)
    vit_base.patch_embed.proj = nn.Conv2d(1, 768, kernel_size=(16, 16), stride=(16, 16))
    vit_base.head = nn.Linear(in_features=768, out_features=out_feat, bias=True)
    torch.cuda.empty_cache()
    checkpoint = torch.load(f'/content/drive/MyDrive/data/Models/{datasetName}/vit_base.pth')
    vit_base.load_state_dict(checkpoint)

    mixer = timm.create_model('mixer_b16_224.goog_in21k_ft_in1k', pretrained=True)
    mixer.stem.proj = nn.Conv2d(1, 768, kernel_size=(16, 16), stride=(16, 16))
    mixer.head = nn.Linear(in_features=768, out_features=out_feat, bias=True)
    torch.cuda.empty_cache()
    checkpoint = torch.load(f'/content/drive/MyDrive/data/Models/{datasetName}/mixer.pth')
    mixer.load_state_dict(checkpoint)

    models_ = [resnet18, resnet50, vit_base, mixer]

    for model in models_:
        attack_dict = {}
        attack_dict['fgsm'] = torchattacks.FGSM(model, eps=8/255)
        attack_dict['bim'] = torchattacks.BIM(model, eps=8/255, alpha=2/255, steps=4)
        attack_dict['pgd'] = torchattacks.PGD(model, eps=8/255, alpha=1/255, steps=4, random_start=True)
        attack_dict['cw'] = torchattacks.CW(model, c=1, kappa=0, steps=20, lr=0.01)

        for name, attk in attack_dict.items():
            img2 = attk(img1, label)

            ssim_value = ssim(img1.numpy()[0][0], img2.squeeze(0).numpy()[0])
            print(ssim_value)

In [29]:
cal_All(esc[0], 50, 'ESC50')

0.621322539261486
0.8007294144150625
0.7842078908830166
1.0000000018687594
0.6279604468767755
0.8216067823774936
0.797706633196687
0.9988706394667558
0.620537140289322
0.6835128611616691
0.7543085658710048
1.0
0.6220936181870146
0.7152210712814139
0.7618809176089838
1.0


In [31]:
import h5py
import torch
from torch.utils.data import TensorDataset

device = "cuda" if torch.cuda.is_available() else "cpu"

data_list = []
label_list = []

# Open the HDF5 file
h5_file_path = '/content/drive/MyDrive/data/AdvData/FSC22.h5'
with h5py.File(h5_file_path, 'r') as hf:
    # Iterate over the samples in the HDF5 file
    for sample_name in hf.keys():
        data = torch.tensor(hf[sample_name]['data'][:])
        label = hf[sample_name]['label'][()]

        # Append data and label to lists
        data_list.append(data)
        label_list.append(label)

fsc = TensorDataset(torch.stack(data_list), torch.tensor(label_list))

In [ ]:
# 1_10101.wav
# class = Fire

In [32]:
cal_All(fsc[0], 27, 'FSC22')

0.8526519081973034
0.9569172642328012
0.9409508417212253
0.9962113671655075
0.8519253464996208
0.9576087284445332
0.9430482381535102
0.9962088565437213
0.8738283478391461
0.9210911743614411
0.9376159420760731
1.0000000060189103
0.85870763971397
0.9478922586967403
0.9387266478266804
0.991838293470986


In [4]:
from torch.utils.data import TensorDataset
import h5py
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

data_list = []
label_list = []

# Open the HDF5 file
h5_file_path = '/content/drive/MyDrive/data/AdvData/DCASE19.h5'
with h5py.File(h5_file_path, 'r') as hf:
    # Iterate over the samples in the HDF5 file
    for sample_name in hf.keys():
        data = torch.tensor(hf[sample_name]['data'][:])
        label = hf[sample_name]['label'][()]

        # Append data and label to lists
        data_list.append(data)
        label_list.append(label)

dcase = TensorDataset(torch.stack(data_list), torch.tensor(label_list))

In [ ]:
# airport-lisbon-1000-40000-a.wav

In [35]:
cal_All(dcase[0], 10, 'DCASE19')

0.8594843033535313
0.9608314139816297
0.9455719091516784
0.9999999923330698
0.8588926796822944
0.9610461216250932
0.9459941572083356
0.9966624355666837
0.8688487953526556
0.959516476774876
0.9452994156654871
0.996911218468413
0.8718841368590087
0.9641339891652809
0.9458713665555678
0.9970229869049493


# Targeted

In [ ]:
from skimage.metrics import structural_similarity as ssim
import numpy as np
import torchattacks

def cal_All(im, out_feat, datasetName):
    img1 = im[0].unsqueeze(0)
    label = im[1].unsqueeze(0)
    resnet18 = timm.create_model('resnet18.a1_in1k', pretrained=True)
    resnet18.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    resnet18.fc = nn.Linear(in_features=512, out_features=out_feat, bias=True)
    torch.cuda.empty_cache()
    checkpoint = torch.load(f'/content/drive/MyDrive/data/Models/{datasetName}/resnet18.pth')
    resnet18.load_state_dict(checkpoint)

    resnet50 = timm.create_model('resnet50.a1_in1k', pretrained=True)
    resnet50.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    resnet50.fc = nn.Linear(in_features=2048, out_features=out_feat, bias=True)
    torch.cuda.empty_cache()
    checkpoint = torch.load(f'/content/drive/MyDrive/data/Models/{datasetName}/resnet50.pth')
    resnet50.load_state_dict(checkpoint)

    vit_base = timm.create_model('vit_base_patch16_224.augreg2_in21k_ft_in1k', pretrained=True)
    vit_base.patch_embed.proj = nn.Conv2d(1, 768, kernel_size=(16, 16), stride=(16, 16))
    vit_base.head = nn.Linear(in_features=768, out_features=out_feat, bias=True)
    torch.cuda.empty_cache()
    checkpoint = torch.load(f'/content/drive/MyDrive/data/Models/{datasetName}/vit_base.pth')
    vit_base.load_state_dict(checkpoint)

    mixer = timm.create_model('mixer_b16_224.goog_in21k_ft_in1k', pretrained=True)
    mixer.stem.proj = nn.Conv2d(1, 768, kernel_size=(16, 16), stride=(16, 16))
    mixer.head = nn.Linear(in_features=768, out_features=out_feat, bias=True)
    torch.cuda.empty_cache()
    checkpoint = torch.load(f'/content/drive/MyDrive/data/Models/{datasetName}/mixer.pth')
    mixer.load_state_dict(checkpoint)

    models_ = [resnet18, resnet50, vit_base, mixer]

    for model in models_:
        attack_dict = {}
        attack_dict['fgsm'] = torchattacks.FGSM(model, eps=8/255)
        attack_dict['fgsm'].set_mode_targeted_by_label()

        attack_dict['bim'] = torchattacks.BIM(model, eps=8/255, alpha=2/255, steps=4)
        attack_dict['bim'].set_mode_targeted_by_label()

        attack_dict['pgd'] = torchattacks.PGD(model, eps=8/255, alpha=1/255, steps=4, random_start=True)
        attack_dict['pgd'].set_mode_targeted_by_label()

        attack_dict['cw'] = torchattacks.CW(model, c=1, kappa=0, steps=20, lr=0.01)
        attack_dict['cw'].set_mode_targeted_by_label()

        for name, attk in attack_dict.items():
            img2 = attk(img1, label)

            ssim_value = ssim(img1.numpy()[0][0], img2.squeeze(0).numpy()[0])
            print(ssim_value)

In [ ]:
import h5py
import torch
from torch.utils.data import TensorDataset

device = "cuda" if torch.cuda.is_available() else "cpu"

data_list = []
label_list = []

# Open the HDF5 file
h5_file_path = '/content/drive/MyDrive/data/AdvData/ESC50.h5'
with h5py.File(h5_file_path, 'r') as hf:
    # Iterate over the samples in the HDF5 file
    for sample_name in hf.keys():
        data = torch.tensor(hf[sample_name]['data'][:])
        label = hf[sample_name]['label'][()]

        # Append data and label to lists
        data_list.append(data)
        label_list.append(label)

esc = TensorDataset(torch.stack(data_list), torch.tensor(label_list))
cal_All(esc[0], 50, 'ESC50')

Attack mode is changed to 'targeted(label)'.
Attack mode is changed to 'targeted(label)'.
Attack mode is changed to 'targeted(label)'.
Attack mode is changed to 'targeted(label)'.
0.6233754622688921
0.8028453057582466
0.7901384914819866
1.0
Attack mode is changed to 'targeted(label)'.
Attack mode is changed to 'targeted(label)'.
Attack mode is changed to 'targeted(label)'.
Attack mode is changed to 'targeted(label)'.
0.6173883390000459
0.8190811953139961
0.7965842792845045
1.0000000018687594
Attack mode is changed to 'targeted(label)'.
Attack mode is changed to 'targeted(label)'.
Attack mode is changed to 'targeted(label)'.
Attack mode is changed to 'targeted(label)'.
0.6387060865855169
0.6837053488259283
0.760695865986228
1.0000000018687594
Attack mode is changed to 'targeted(label)'.
Attack mode is changed to 'targeted(label)'.
Attack mode is changed to 'targeted(label)'.
Attack mode is changed to 'targeted(label)'.
0.6251387941455783
0.7764016357770613
0.7670598840479718
1.000000001

In [ ]:
import h5py
import torch
from torch.utils.data import TensorDataset

device = "cuda" if torch.cuda.is_available() else "cpu"

data_list = []
label_list = []

# Open the HDF5 file
h5_file_path = '/content/drive/MyDrive/data/AdvData/FSC22.h5'
with h5py.File(h5_file_path, 'r') as hf:
    # Iterate over the samples in the HDF5 file
    for sample_name in hf.keys():
        data = torch.tensor(hf[sample_name]['data'][:])
        label = hf[sample_name]['label'][()]

        # Append data and label to lists
        data_list.append(data)
        label_list.append(label)

fsc = TensorDataset(torch.stack(data_list), torch.tensor(label_list))
cal_All(fsc[0], 27, 'FSC22')

Attack mode is changed to 'targeted(label)'.
Attack mode is changed to 'targeted(label)'.
Attack mode is changed to 'targeted(label)'.
Attack mode is changed to 'targeted(label)'.
0.8531919338468168
0.9596758009322655
0.9426149029732152
1.0000000060189103
Attack mode is changed to 'targeted(label)'.
Attack mode is changed to 'targeted(label)'.
Attack mode is changed to 'targeted(label)'.
Attack mode is changed to 'targeted(label)'.
0.8531919985673472
0.9588551334676172
0.9426523437266517
1.0000000060189103
Attack mode is changed to 'targeted(label)'.
Attack mode is changed to 'targeted(label)'.
Attack mode is changed to 'targeted(label)'.
Attack mode is changed to 'targeted(label)'.
0.8762825560628962
0.9206379457183201
0.9386175317378891
0.9982814576519768
Attack mode is changed to 'targeted(label)'.
Attack mode is changed to 'targeted(label)'.
Attack mode is changed to 'targeted(label)'.
Attack mode is changed to 'targeted(label)'.
0.8594856938334208
0.9595800777916835
0.942283343749

In [ ]:
from torch.utils.data import TensorDataset
import h5py
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

data_list = []
label_list = []

# Open the HDF5 file
h5_file_path = '/content/drive/MyDrive/data/AdvData/DCASE19.h5'
with h5py.File(h5_file_path, 'r') as hf:
    # Iterate over the samples in the HDF5 file
    for sample_name in hf.keys():
        data = torch.tensor(hf[sample_name]['data'][:])
        label = hf[sample_name]['label'][()]

        # Append data and label to lists
        data_list.append(data)
        label_list.append(label)

dcase = TensorDataset(torch.stack(data_list), torch.tensor(label_list))
cal_All(dcase[0], 10, 'DCASE19')

Attack mode is changed to 'targeted(label)'.
Attack mode is changed to 'targeted(label)'.
Attack mode is changed to 'targeted(label)'.
Attack mode is changed to 'targeted(label)'.
0.8590617362218878
0.9612016393942981
0.9463096768737552
0.993146528135184
Attack mode is changed to 'targeted(label)'.
Attack mode is changed to 'targeted(label)'.
Attack mode is changed to 'targeted(label)'.
Attack mode is changed to 'targeted(label)'.
0.8585614229332728
0.9614430526828798
0.9461031682302069
0.9999999923330698
Attack mode is changed to 'targeted(label)'.
Attack mode is changed to 'targeted(label)'.
Attack mode is changed to 'targeted(label)'.
Attack mode is changed to 'targeted(label)'.
0.8665396744188922
0.9699784232998484
0.9457419935992326
0.9999999923330698
Attack mode is changed to 'targeted(label)'.
Attack mode is changed to 'targeted(label)'.
Attack mode is changed to 'targeted(label)'.
Attack mode is changed to 'targeted(label)'.
0.8717203711848319
0.9677147174845259
0.9460279239845